In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-18 19:43:32--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.80MB/s    in 0.2s    

2021-09-18 19:43:33 (5.80 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles

# Extract dataset used in deliverable 1

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

In [5]:
# Recreate vine table from deliverable 1
# Create the vine_table DataFrame
df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28HBXXO1UEVJT|          5|            0|          0|   N|                N|
| RZKRFS2UUMFFU|          5|           15|         15|   N|                Y|
|R2WAU9MD9K6JQA|          3|            6|          8|   N|                N|
|R36SCTKYTVPZPC|          5|           10|         11|   N|                Y|
|R10BM6JUOJX27Q|          3|            0|          0|   Y|                N|
| RCLZ5OKZNUSY4|          5|            0|          0|   N|                Y|
|R1S65DJYEI89G4|          4|            8|         17|   N|                N|
|R3KQYBQOLYDETV|          4|            2|          2|   N|                N|
|R3QV8K7CSU8K2W|          5|            0|          0|   N|                N|
|R3W5A1WUGO5VQ0|          4|            0|          1|   N|     

In [6]:
# filter DataFrame for total_votes >= 20
df_filtered = df.filter(df.total_votes >= 20)
df_filtered.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R18VIM840CEFRP|          1|           16|        105|   N|                N|
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|
| RBQZC5A3TSWT5|          5|           11|         22|   N|                Y|
| RW00TDPV9U93E|          1|           35|         76|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [7]:
# Filter DataFrame for helpful_votes/total votes >= 0.5
df_helpful = df_filtered.filter((df_filtered.helpful_votes / df_filtered.total_votes) >= 0.5)
df_helpful.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|
| RBQZC5A3TSWT5|          5|           11|         22|   N|                Y|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [8]:
# Filter for paid vine
df_paid = df_helpful.filter(df_helpful.vine == 'Y')
df_paid.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2Z8083V8ZFQVZ|          3|           82|         87|   Y|                N|
| RK25TAO4GGS5G|          3|           22|         24|   Y|                N|
|R2SJQG3C6KY0M8|          3|           18|         20|   Y|                N|
|R12CHC0CB2WASU|          5|           43|         48|   Y|                N|
|R2RY328TIDXMTE|          2|           50|         61|   Y|                N|
|R3K8OQU4PEP8W7|          4|           40|         46|   Y|                N|
|R1JI1A7JT99538|          3|           28|         40|   Y|                N|
|R247K3K5SVPJUL|          3|           15|         20|   Y|                N|
|R27UEUDUYPLXZC|          5|           20|         24|   Y|                N|
|R2QQDNBYDIEL9F|          5|           35|         41|   Y|     

In [9]:
# Filter for unpaid
df_unpaid = df_helpful.filter(df_helpful.vine == 'N')
df_unpaid.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|
| RBQZC5A3TSWT5|          5|           11|         22|   N|                Y|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [11]:
# total number of reviews (with those having votes >=20 and ratio of helpful to total votes >= 0.5)
total number of reviews = df_helpful.count()

337176

In [18]:
# total number of 5 star reviews
total_five = df_helpful.filter(df_helpful.star_rating == 5).count()
total_five

170404

In [23]:
#number of paid reviews 
paid_total = df_paid.count()

#number of paid 5 star reviews
paid_five = df_paid.filter(df_paid.star_rating == 5).count()

#percentage paid 5 star reviews
paid_five_percent = (paid_five/paid_total)*100
paid_five_percent

33.5494666387785

In [24]:
#number of paid reviews 
unpaid_total = df_unpaid.count()

#number of paid 5 star reviews
unpaid_five = df_unpaid.filter(df_unpaid.star_rating == 5).count()

#percentage paid 5 star reviews
unpaid_five_percent = (unpaid_five/unpaid_total)*100
unpaid_five_percent

50.782954015553784